In [1]:
import torch
from torch_geometric.data import HeteroData
import pandas as pd
import sys
# Import des fonctions et du modèle
from src.utils import load_and_prepare_real_data, run_community_detection
from src.hetero import HeteroGNN

print("--- Démarrage du Pipeline GML Géospatial ---")

--- Démarrage du Pipeline GML Géospatial ---


In [12]:
import pandas as pd
import numpy as np
import geopandas as gpd
import torch
import torch.nn.functional as F
import networkx as nx
import community as community_louvain 
import sklearn
import torch_geometric
import shapely

In [14]:
print(pd.__version__)
print(np.__version__)
print(gpd.__version__)
print(nx.__version__)
print(sklearn.__version__)
print(shapely.__version__)

2.3.3
2.3.3
1.1.1
3.5
1.7.2
2.1.2


In [2]:
# Étape 1 : Préparation des données et du graphe
try:
    adr_x, bat_x, par_x, edge_index_bp, edge_attr_bp, edge_index_ab, edge_index_ba, bat_map, par_map, adr_map = load_and_prepare_real_data()
except Exception as e:
    print(f"\nERREUR CRITIQUE PENDANT LE CHARGEMENT/PRÉPARATION DES DONNÉES : {e}")
    # Si l'erreur persiste, vérifier si le chemin 'gml.zip/...' est correct
    exit()

--- 1. Chargement et Préparation des Données Réelles ---
Calcul du point représentatif des Parcelles (point garanti dans la géométrie)...
Exécution de la jointure spatiale (Parcelle.ReprensentativePoint WITHIN Zone PLU)...
Calcul de la relation Adresse -> Bâtiment (sjoin_nearest)...
Création de la relation inverse Bâtiment -> Adresse...


In [10]:
# Construction du Graphe Hétérogène (PyG HeteroData)
data = HeteroData()
data['adresse'].x = adr_x
data['bâtiment'].x = bat_x
data['parcelle'].x = par_x

In [12]:
# Relations
# data.edge_index_dict = {
#     ('adresse', 'accès', 'bâtiment'): edge_index_ab,
#     ('bâtiment', 'dessert', 'adresse'): edge_index_ba, # <-- L'ajout crucial
#     ('bâtiment', 'appartient', 'parcelle'): edge_index_bp
# }
data['adresse', 'accès', 'bâtiment'].edge_index = edge_index_ab
data['bâtiment', 'dessert', 'adresse'].edge_index = edge_index_ba
data['bâtiment', 'appartient', 'parcelle'].edge_index = edge_index_bp
#data['bâtiment', 'appartient', 'parcelle'].edge_attr = edge_attr_bp 

print(f"\nStructure du Graphe créée. Nœuds: {len(data['bâtiment'].x)} | Arêtes B->P: {data['bâtiment', 'appartient', 'parcelle'].num_edges}")


Structure du Graphe créée. Nœuds: 691551 | Arêtes B->P: 449229


In [13]:
data.metadata()

(['adresse', 'bâtiment', 'parcelle'],
 [('adresse', 'accès', 'bâtiment'),
  ('bâtiment', 'dessert', 'adresse'),
  ('bâtiment', 'appartient', 'parcelle')])

In [14]:
# Étape 2 : Passage de Messages GNN (Création d'Embeddings)
print("\n--- 2. Passage de Messages GNN (Création d'Embeddings) ---")
    
HIDDEN_CHANNELS = 64
OUT_CHANNELS = 32
metadata = data.metadata() 
bat_embeddings = None


--- 2. Passage de Messages GNN (Création d'Embeddings) ---


In [15]:
node_feature_sizes = {node_type: data[node_type].x.shape[1] for node_type in data.node_types}

In [16]:
node_feature_sizes

{'adresse': 2, 'bâtiment': 2, 'parcelle': 650}

In [17]:
try:
    model = HeteroGNN(
        hidden_channels=64, 
        out_channels=32, 
        num_layers=2,
        node_feature_sizes=node_feature_sizes  # <-- On passe le dictionnaire ici
    )
    
    # Appel du modèle SANS le dictionnaire edge_attr_dict
    x_dict = model(data.x_dict, data.edge_index_dict)
    
    bat_embeddings = x_dict['bâtiment']
    print(f"Embeddings générés. Shape: {bat_embeddings.shape}")

except Exception as e:
    print(f"\nERREUR D'EXÉCUTION DU MODÈLE GNN : {e}")
    print("Vérifiez la taille des tenseurs x et le bon type (float) des features.")

Embeddings générés. Shape: torch.Size([691551, 32])


In [ ]:
# Étape 3 : Détection de Communautés (Louvain)
if bat_embeddings is not None:
    print("\n--- 3. Détection des Entités Cohérentes (Louvain) ---")
    
    partition = run_community_detection(bat_embeddings, similarity_threshold=0.7) 
    
    if partition is not None:
        
        # Affichage des résultats concrets
        bat_id_list = list(bat_map.keys())
        df_bat_result = pd.DataFrame({'bat_id': bat_id_list, 'community_id': [partition.get(i) for i in range(len(bat_id_list))]})
        
        num_communities = df_bat_result['community_id'].nunique()
        
        print("\n--- RÉSULTATS DÉTAILLÉS ---")
        print(f"Total de {num_communities} 'Cités' (communautés) découvertes.")
        
        print("\nTaille des Communautés (Nombre de Bâtiments) :")
        print(df_bat_result['community_id'].value_counts().head(5))
        
        plus_grande_communaute = df_bat_result['community_id'].value_counts().index[0]
        exemples_ids = df_bat_result[df_bat_result['community_id'] == plus_grande_communaute]['bat_id'].head(5).tolist()
        print(f"\nExemples de Bâtiments dans la Cité la plus grande (ID {plus_grande_communaute}): {exemples_ids}")

print("\nPipeline terminé.")


--- 3. Détection des Entités Cohérentes (Louvain) ---
